In [61]:
import pandas as pd
import csv

In [62]:
df = pd.read_csv('database.csv')
df.head()

,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_sig_str_percentage,blueCorner_sig_str_percentage,redCorner_total_str,blueCorner_total_str,redCorner_takedowns,blueCorner_takedowns,redCorner_takedown_percentage,blueCorner_takedown_percentage,redCorner_subs_attempted,blueCorner_subs_attempted
0,Leon Edwards vs Colby Covington,Leon Edwards,Colby Covington,Leon Edwards,UFC 296: Edwards vs. Covington,Herb Dean,Decision - Unanimous,12.16.2023,T-Mobile Arena,yes,...,52%,34%,65 of 116,109 of 194,2 of 3,2 of 10,66%,20%,2.0,0.0
1,Alexandre Pantoja vs Brandon Royval,Alexandre Pantoja,Brandon Royval,Alexandre Pantoja,UFC 296: Edwards vs. Covington,Marc Goddard,Decision - Unanimous,12.16.2023,T-Mobile Arena,yes,...,60%,54%,126 of 195,281 of 390,8 of 14,0 of 1,57%,0%,1.0,0.0
2,Shavkat Rakhmonov vs Stephen Thompson,Shavkat Rakhmonov,Stephen Thompson,Shavkat Rakhmonov,UFC 296: Edwards vs. Covington,Jason Herzog,Submission,12.16.2023,T-Mobile Arena,no,...,77%,64%,73 of 88,26 of 37,1 of 5,0 of 0,20%,---,2.0,0.0
3,Paddy Pimblett vs Tony Ferguson,Tony Ferguson,Paddy Pimblett,Paddy Pimblett,UFC 296: Edwards vs. Covington,Mark Smith,Decision - Unanimous,12.16.2023,T-Mobile Arena,no,...,59%,64%,90 of 134,151 of 217,0 of 0,1 of 4,---,25%,0.0,0.0
4,Josh Emmett vs Bryce Mitchell,Josh Emmett,Bryce Mitchell,Josh Emmett,UFC 296: Edwards vs. Covington,Herb Dean,KO/TKO,12.16.2023,T-Mobile Arena,no,...,20%,0%,2 of 10,0 of 3,0 of 0,0 of 0,---,---,0.0,0.0


In [63]:
print(len(df))

7457


In [64]:
#Dropping all rows where red and blue corners are not recorded correctly
#Since this will effect the accuracy of the model I am removing it for now
#Will try and find a credible source to fix this at a later date
for index, row in df.iterrows():
    if(index>6150):
        df.drop(index, inplace=True)

In [65]:
df2 = pd.read_csv('ufc_fighters_statistics.csv')
df2.head()

,name,wins,losses,draws,height,weight,reach,stance,dOB,sig_strikes_landed_per_min,sig_striking_accuracy_%,sig_strike_absorbed_per_min,sig_strike_defense(%_of_sig_strikes_not_landed_by_opponent),takedown_average(average_takedown_landed_per_fifteen_min),takedown_accuracy_%,takedown_defense(%_of_opponent_takedown_not_landed),sub_average(average_subs_attempted_per_15_mins)
0,Tom Aaron,5,3,0,--,155 lbs.,--,NaN,"Jul 13, 1978",0.00,0%,0.00,0%,0.00,0%,0%,0.0
1,Danny Abbadi,4,6,0,"5' 11""",155 lbs.,--,Orthodox,"Jul 03, 1983",3.29,38%,4.41,57%,0.00,0%,77%,0.0
2,Nariman Abbasov,28,4,0,"5' 8""",155 lbs.,"66""",Orthodox,"Feb 01, 1994",3.00,20%,5.67,46%,0.00,0%,66%,0.0
3,David Abbott,10,15,0,"6' 0""",265 lbs.,--,Switch,--,1.35,30%,3.55,38%,1.07,33%,66%,0.0
4,Hamdy Abdelwahab,5,0,0 (1 NC),"6' 2""",264 lbs.,"72""",Southpaw,"Jan 22, 1993",3.87,52%,3.13,59%,3.00,75%,0%,0.0


In [66]:
#find special characters in df
names = (df2['name'].values)
chars2rep = []
for name in names:
    if(isinstance(name, str)):
        for char in name:
            if char.lower() not in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']:
                chars2rep.append(char)
            if char.upper() not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', "Q", 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']:
                chars2rep.append(char)
chars2rep = list(set(chars2rep))
for char in chars2rep:
    print(char)

-
'
.


In [67]:
for index, row in df2.iterrows():
    name = df2['name']
    if(isinstance(name, str)):
        if("'" in name):
            df2.loc[index, 'name'] = name.replace("'", '')
        if('.' in name):
            df2.loc[index, 'name'] = name.replace('.', '')
        if('-' in name):
            df2.loc[index, 'name'] = name.replace('-', ' ')

In [71]:
count = 0
matched = []
notMatched = []
for index2, row2 in df.iterrows():
    for index, row in df2.iterrows():
        name = str(row['name']).replace(" ", "").lower()
        redCorner = str(row2['redCorner']).replace(" ", "").lower()
        blueCorner = str(row2['blueCorner']).replace(" ", "")
        if(name in redCorner or redCorner in name):
            matched.append(row2['redCorner'])
        elif(name in blueCorner or blueCorner in name):
            matched.append(row2['blueCorner'])
        else:
            name = ''.join(sorted(str(row['name']).replace(" ", "").lower()))
            redCorner = ''.join(sorted(str(row2['redCorner']).replace(" ", "").lower()))
            blueCorner = ''.join(sorted(str(row2['blueCorner']).replace(" ", "")))
            if(name in redCorner or redCorner in name):
                matched.append(row2['redCorner'])
            if(name in blueCorner or blueCorner in name):
                matched.append(row2['blueCorner'])
    if(row2['redCorner'] not in matched):
        notMatched.append(row2['redCorner'])
    if(row2['blueCorner'] not in matched):
        notMatched.append(row2['blueCorner'])

notMatched = list(set(notMatched))
for name in notMatched:
    print(name)

Mike Pierce
Roger Zapata
Wagner Campos
Richie Smullen
Mario Bautista
Karlos Vemola
Will Brooks
JC Cottrell
Thomas Almeida
Julio Paulino
Viviane Araujo
Ben Askren
Nick Pace
Leo Kuntz
Kyle Nelson
Artem Lobov
Rashid Magomedov
Hector Sandoval
Dominique Steele
Charlie Valencia
Cody Brundage
Patrick Walsh
Chepe Mariscal
Jianping Yang
Mike Rio
Stephen Thompson
Brian Foster
Mike Mathetha
Iasmin Lucindo
Aleksei Oleinik
John Salter
Eric Spicely
Montserrat Rendon
Raquel Pennington
Zarrukh Adashev
Clay Harvison
Danyelle Wolf
Rachael Ostovich
Curt Warburton
Azunna Anyanwu
Andre Winner
Bart Palaszewski
Guto Inocente
John Cofer
Dhiego Lima
Dave Galera
Masanori Kanehara
Domingo Pilarte
Bryan Caraway
Lucas Almeida
Court McGee
Mandy Bohm
Roque Martinez
Daniel Pineda
Rogerio Nogueira
Roger Narvaez
Joanderson Brito
Josh Shockley
Cory McKenna
Jamall Emmers
Muin Gafurov
Jamal Pogues
Bruna Brasil
Ronaldo Souza
Alex Chambers
Shamar Bailey
Jason Brilz
Gabe Green
Markus Perez
Jessin Ayari
Jarjis Danho
Salim Tou